# Swaps and spawtions project
This project contains functions to price interest rates swaps, and options on those swaps (swaptions). To do that, it models floating rates with a self-made mean-reverting binomial tree model. All the steps are explained with markdowns.

In [ ]:
import numpy as np

# This code cell is for all intermediary simple functions necessary for calculations. For now, the only one we can modify
# depending on the model we want to use is y(t). A different model could be used also for r(t), but this is a good approximation.

def y(t):
    '''
    Yield curve. We'll use it to calculate discount factors and risk-free rates.
    '''
    # In this case, we will use constant rate, but we can adapt it to a different yield curve
    return 0.04

def d_f(tf, ti):
    '''
    Discount value of money through yield curve of zero coupon bonds.

    Parameters:
        tf (float): Point in time where the amount of money exists.
        ti (float): Point in time in terms of which we want to calculate the value of the tf money.

    Returns:
        float: Discount factor, less than 1 if tf > ti.
    '''
    return (1 + y(ti)) ** ti / (1 + y(tf)) ** tf

def diff(f, x0, dx=1e-7):
    '''
    Derivative of function f(x) at point x0.
    '''
    return (f(x0 + dx) - f(x0)) / dx

def r(t):
    '''
    Risk-free rate at time t.
    '''
    # (d / dt)(t * y(t)), we use product rule:
    return y(t) + t * diff(y, t)

In [175]:
# Code cell to create plots for markdowns, ignore

import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import os
os.remove("normal_rate.png")
os.remove("normal_threshold.png")

mu, sigma = 0, 1  
x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
n = norm.pdf(x, mu, sigma)

# Define the two special x-axis points
special_x = [-2, 0, 1]  # Example values for r_n and r_{n+1}

plt.figure()
# Plot Normal Distribution
plt.plot(x, n, label=r'$\mathcal{N}\left[(1 - \alpha)\cdot r_i(t) + \alpha\cdot r_{mean}, \sigma \right]$', color='blue')
# Remove default numerical ticks
plt.xticks(special_x, ["$r_{mean}$", r"$\mu$", r"$r(t)$"])  # Custom labels
plt.yticks([])
# Labels and Title
plt.xlabel("$r(t + 1)$")
plt.ylabel("pdf")
plt.title("Normal Distribution with Custom X-axis Labels")
plt.grid(True)
plt.savefig("normal_rate.png")  # Save the plot as an image
plt.close()  # 🔹 Prevents it from displaying


plt.figure()
# Plot Normal Distribution
plt.plot(x, n, label=r'$\mathcal{N}\left[(1 - \alpha)\cdot r_i(t) + \alpha\cdot r_{mean}, \sigma \right]$', color='blue')
# Remove default numerical ticks
plt.xticks(special_x, ["$r_{mean}$", r"$\mu$", r"$r_i(t)$"])  # Custom labels
plt.yticks([])
# Highlight area to the right of r_t
x_fill = np.linspace(1, mu + 4*sigma, 500)  # Range for shaded region
y_fill = norm.pdf(x_fill, mu, sigma)
plt.fill_between(x_fill, y_fill, alpha=0.3, color='red', label=r"$p$")
# Labels and Title
plt.xlabel("$r(t + 1)$")
plt.ylabel("pdf")
plt.title("Normal Distribution highlighting probability of $r(t + 1)$ above threshold, $p$")
plt.legend(loc = "upper right")
plt.grid(True)
plt.savefig("normal_threshold.png")  # Save the plot as an image
plt.close()  # Prevents it from displaying




## Modelling the evolution of the floating rate

In this section, we implement a binomial tree model to simulate the evolution of the floating interest rate over time. A binomial tree is a simple tool for modeling stochastic processes, where the rate can move up or down at each step. In our model, the rate moves by a constant factor up, $u$, and a constant factor down, $d = \frac{1}{u}$. However, one of the most important characteristics of interest rates in reality is mean reversion—the tendency for rates to move toward a long-term average rather than drift indefinitely in one direction.

To incorporate mean reversion into our model, we keep the traditional binomial tree structure but modify the way we calculate the probability of an upward move. We developed a way to introduce it based on the normal discribution function, instead of using a fixed risk-neutral probability formula. This is how it works:

Suppose we find ourselves at the $i^{th}$ node of time step $t$. Here, the floating rate $r(t)$ is $r_i(t)$. Then, as we move one time step forward, we say that the rate in the next time step, $r(t + 1)$ follows a normal distribution where the average is a weighted mean of $r(t)$ and $r_{mean}$, and the standard deviation is $\sigma$:
$$
r(t + 1) \sim \mathcal{N}(\mu, \sigma), \\
\text{where } \mu = (1 - \alpha)\cdot r(t) + \alpha\cdot r_{mean}
$$
The following plot shows the probability distribution of $r(t + 1)$ when $r(t) > r_{mean}$.

<p align="center">
    <img src="normal_rate.png" width="600">
</p>

$r_{mean}$, $\alpha$ and $\sigma$ are provided as parameters to the model. We call $\alpha$ the mean reversion speed. This way, the rate reverts to the mean rate at each time step, with a bigger effect the further away it is from the mean.

We must combine this model with the tree model, in which an upward movement with probability $p$ takes us to $r_i(t + 1) = u \cdot r_i(t)$, and a downward movement with probability $1 - p$ takes us to $r_{i + 1}(t + 1) = \frac{1}{u} \cdot r_i(t)$. In order to do that, we will define a rate threashold in the normal distribution such that the probability of an upward movement, $p$, is the probability of the rate being above that threashold. We take as a threashold the geometric mean between $r_i(t + 1)$ and $r_{i + 1}(t + 1)$:
$$
\text{Mean}_{geom}\left[r_i(t + 1), r_{i + 1}(t + 1)\right] = \sqrt{r_i(t + 1) \cdot r_{i + 1}(t + 1)} = \sqrt{u \cdot r_i(t) \cdot \frac{1}{u} \cdot r_i(t)} = r_i(t) \\
\text{Mean}_{geom}\left[r_i(t + 1), r_{i + 1}(t + 1)\right] = r_i(t)
$$

<p align="center">
    <img src="normal_threshold.png" width="700">
</p>


Below is the Python function that constructs this mean-reverting binomial tree for interest rates.



In [176]:
import numpy as np
from scipy.stats import norm


def binomial_interest_rate_tree(r0, u, N, r_mean = None, mean_reversion_speed = 0.2, std_dev = 0.5):
    '''
    Generate a binomial tree for interest rates.

    Parameters:
        r0 (float): Initial short rate.
        u (float): Upward movement factor.
        N (int): Number of time steps.
        r_mean (float): Long-term mean interest rate toward which the rates revert.
        mean_reversion_speed (float): Strength of mean reversion.
        std_dev (float): Standard deviation of interest rate changes in the next time step, used to calcilate the probability of an upward move.

    Returns:
        tuple:
            1. np.ndarray: A lower-triangular matrix representing the interest rate tree.
            2. np.ndarray: A matrix of the same shape representing the probability an upward movement at each node.
    '''
    # If no r_mean is provided, default to r0
    if r_mean is None:
        r_mean = r0
    # Set downward movement factor
    d = 1 / u
    # Initialize the interest rate tree matrix and probability tree
    tree = np.zeros((N + 1, N + 1))
    probabilities = np.zeros((N + 1, N + 1))
    # Populate the tree
    for i in range(N + 1):
        for j in range(i + 1):
            tree[j, i] = r0 * (u ** (i - j)) * (d ** j)  # Interest rate at node (j, i)
            probabilities[j, i] = (1 - norm.cdf(tree[j, i], loc = (1 - mean_reversion_speed) * tree[j, i] + mean_reversion_speed * r_mean, scale = std_dev))  # p(i, u, d)

    return tree, probabilities



# Example parameters
r0 = 4
u = 1.1
N = 5

interest_rate_tree = binomial_interest_rate_tree(r0, u, N)

print(interest_rate_tree[0])
print(interest_rate_tree[1])

[[4.         4.4        4.84       5.324      5.8564     6.44204   ]
 [0.         3.63636364 4.         4.4        4.84       5.324     ]
 [0.         0.         3.30578512 3.63636364 4.         4.4       ]
 [0.         0.         0.         3.0052592  3.30578512 3.63636364]
 [0.         0.         0.         0.         2.73205382 3.0052592 ]
 [0.         0.         0.         0.         0.         2.48368529]]
[[0.5        0.43644054 0.36843543 0.29819465 0.22887406 0.16433013]
 [0.         0.557824   0.5        0.43644054 0.36843543 0.29819465]
 [0.         0.         0.60937328 0.557824   0.5        0.43644054]
 [0.         0.         0.         0.65464669 0.60937328 0.557824  ]
 [0.         0.         0.         0.         0.6939852  0.65464669]
 [0.         0.         0.         0.         0.         0.7279172 ]]


# Pricing the swaption

A swaption (swap option) is a financial derivative that gives the holder the right, but not the obligation, to enter into an interest rate swap at a predetermined fixed rate. In our case, we consider a swaption that allows the holder to enter into a fixed-rate swap at any point before the contract’s maturity. This flexibility introduces a decision-making aspect at each time step: either exercise the swaption and enter the fixed-rate swap or continue holding the swaption in the hope of better future conditions.

### Step 1: Valuation at Maturity
To determine the value of the swaption, we first calculate its value for all the nodes at maturity. At this point, the holder can no longer defer their decision, so the swaption's value is given by the maximum of:
- The value of the underlying swap at that node.
- Zero, if the holder decides not to exercise the swaption.
For this, we still need to calculate the value of the underlying swap at that node, but we will explain how to do this later.

Mathematically, at maturity (final time step $N$), the value of the swaption at node $(i, N)$ is:
$$
V(i, N) = \text{max}(S(i, N), 0)
$$
where $S(i, N)$ is the value of the swap at that node, and $V(i, N)$ is the value of the swaption.

### Step 2: Backwards induction
After computing the swaption value at maturity, we use backward induction to determine its value at earlier time steps. At each node , the value of the swaption is the maximum of:
- The immediate exercise value of the underlying the swap at that node.
- The expected value of the swaption if held until the next time step.
$$
V(i, t) = \text{max}(S(i, t), E\left[V(t + 1) | V(i, t)\right])
$$
This expected value we calculate with the probabilities given by the probability tree:
$$
E\left[V(t + 1) | V(i, t)\right] = p(i, t) \cdot V(i, t + 1) + (1 - p(i, t)) \cdot V(i + 1, t + 1)
$$
where $p(i, t)$ is the probability of upward movement found in the node $(i, t)$ of the probability tree.

We can apply backwards induction indefinitely until we reach $t = 0$, at which point we have calculated the current value of the swaption.

### Calculating the value of executing the swap at each node
The value of the swap $S(i, t)$ at each node is the sum of all the discounted cashflows from that node onwards. We know exactly the cashflow at that node, but for future nodes, we have to consider the expected value, since we don't know which path in the tree the interest rate is gonna take. Thus, we must sum:
- The net cash flow at the current node, given by the difference between the floating rate (from the binomial tree) and the fixed rate agreed upon in the swap contract.
- The expected discounted value of the swap in future time steps.
$$
S(i, t) = D(t, 0)\frac{r(i, t), r_{fixed}}{100} + p(i, t) \cdot S(i, t + 1) + (1 - p(i, t)) \cdot S(i + , t + 1)
$$
where $D(t, 0)$ is the discount factor from time $t$ to time 0, which we calculate with our function d(tf, ti).

We can price the swap for the whole tree, again, starting from the price at maturity and applying backwards induction. The value at maturity is just the discounted cashflow at that node, since after that the contrat expires, no more payments.

### Note
Note that when we calculate the value of the swaption at each node, we don't apply discount factor from that time to the present, since it has already been applied in the calculation of the swap. Our resulting trees which contain the prices of the swaption and swap at each node are all discounted to present day value.

In [177]:
import numpy as np


def swap_values(rates_prob_matrix, fixed_rate):
    '''
    Situation: We enter a floating rate interest payments contract at t = 0 with repayment due at t = (nº of time steps in rates_matrix). We
    are considering the swap that swaps our interest rate to a fixed one of (fixed_rate)% for the remainder of the
    contract, which ends at t = (nº of time steps in rates_matrix).
    '''
    '''
    Calculate the value of an interest rate swap at each node of a binomial tree, in terms of value at t = 0.

    Parameters:
        rates_prob_matrix (tuple): Tuple containing the following two matricies (ordered):
            1. np.ndarray: Binomial tree matrix representing the evolution of interest rates (e.g., LIBOR) over time, entries as percentages.
            2. np.ndarray: A matrix of the same shape representing the risk-neutral probability of upward movement at each node.
        fixed_rate (float): The fixed interest rate in the swap agreement, as a percentage.
        p (float): Risk-neutral probability of an upward movement in the binomial tree, value between 0 and 1.

    Returns:
        numpy.ndarray: Binomial tree matrix containing the value of the swap at each node, in terms of present value (t = 0).
    '''
    # Separate rates matrix and risk-neutral probabilities matrix
    rates_matrix, prob = rates_prob_matrix
    # Number of time steps in the binomial tree
    N = rates_matrix.shape[0] - 1
    # Initialize a matrix to store the swap values at each node
    values = np.zeros((N + 1, N + 1))

    # Step 1: Calculate the swap values at the final time step (maturity)
    for i in range(N + 1):
        # At maturity, the swap value is the discounted net cash flow
        values[i, N] = d_f(N, 0) * (rates_matrix[i, N] - fixed_rate) / 100

    # Step 2: Work backward through the binomial tree to calculate swap values at earlier nodes
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            # The swap value at the current node is the sum of
            # 1. The net cash flow at the current node
            # 2. The discounted expected future value of the swap
            values[j, i] = d_f(i, 0) * (rates_matrix[j, i] - fixed_rate) / 100 + prob[j, i] * values[j, i + 1] + (1 - prob[j, i]) * values[j + 1, i + 1]

    return values


def swaption_values(rates_prob_matrix, fixed_rate):
    '''
    Situation: We enter a floating rate interest payments contract at t = 0 with repayment due at t = (nº of time steps in rates_matrix).
    We are considering the swaption that swaps our interest rate to a fixed one of (fixed_rate)% for the remainder of the contract, which
    ends at t = (nº of time steps in rates_matrix). We can execute the swaption just before any time step after we buy it.
    '''
    '''
    Calculate the value of an interest rate swaption at each node of a binomial tree, in terms of value at t = 0.

    Parameters:
        rates_prob_matrix (tuple): Tuple containing the following two matricies (ordered):
            1. np.ndarray: Binomial tree matrix representing the evolution of interest rates (e.g., LIBOR) over time, entries as percentages.
            2. np.ndarray: A matrix of the same shape representing the risk-neutral probability of upward movement at each node.
        fixed_rate (float): The fixed interest rate in the swap agreement, as a percentage.

    Returns:
        numpy.ndarray: Binomial tree matrix containing the value of the swaption at each node, in terms of present value (t = 0).
    '''
    # Separate rates matrix and risk-neutral probabilities matrix
    rates_matrix, prob = rates_prob_matrix
    # Number of time steps in the binomial tree
    N = rates_matrix.shape[0] - 1
    # Initialize a matrix to store the swaption values at each node
    values = np.zeros((N + 1, N + 1))
    # Obtain the matrix with the value of the swap at each node for future calculations
    swap_value_matrix = swap_values(rates_prob_matrix, fixed_rate)

    # Step 1: Calculate the swaption values at the final time step (maturity)
    for i in range(N + 1):
        if rates_matrix[i, N] > fixed_rate:
        # At maturity, the swaption value is the maximum of the discounted net cash flow and 0
            values[i, N] = d_f(N, 0) * (rates_matrix[i, N] - fixed_rate) / 100

    # Step 2: Work backward through the binomial tree to calculate swaption values at earlier nodes
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            # The swaption value at the current node is the maximum of
            # 1. The value of the swaption if executed then
            # 2. The expected value of the swaption if held without executing
            values[j, i] = max([
                swap_value_matrix[j, i] ,
                prob[j, i] * values[j, i + 1] + (1 - prob[j, i]) * values[j + 1, i + 1]
                ])

    return values


# Example parameters
r0 = 4
u = 1.1
N = 5
option_rate = 4
interest_rate_tree = binomial_interest_rate_tree(r0, u, N)

print(np.round(swap_values(interest_rate_tree, option_rate), 5))
print(np.round(swaption_values(interest_rate_tree, option_rate), 5))

[[ 0.00147  0.01501  0.02466  0.02968  0.02885  0.02007]
 [ 0.      -0.01206  0.00072  0.00943  0.01327  0.01088]
 [ 0.       0.      -0.02028 -0.00799  0.00015  0.00329]
 [ 0.       0.       0.      -0.02303 -0.01095 -0.00299]
 [ 0.       0.       0.       0.      -0.02033 -0.00818]
 [ 0.       0.       0.       0.       0.      -0.01246]]
[[0.00907 0.01501 0.02466 0.02968 0.02885 0.02007]
 [0.      0.00313 0.00517 0.00943 0.01327 0.01088]
 [0.      0.      0.00056 0.00092 0.00164 0.00329]
 [0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.     ]]


In [178]:
import numpy as np

def option_values(rates_prob_matrix, loan_y, option_rate):
    '''
    Situation: We are considering an option to enter a fixed-rate loan with an interest rate of (option_rate), lasting (loan_y)
    years with annual payments. The option can be exercised at any time before t = (nº of time steps in rates_matrix), after
    which the right expires. To determine its worth, we compare it to taking out an almost identical fixed-rate loan contract
    with interest rate determined by the floating rate that would otherwise apply at that time.
    '''
    '''
    Generate value of option at each node, in terms of value at time t = 0.

    Parameters:
        rates_prob_matrix (tuple): Tuple containing the following two matricies (ordered):
            1. np.ndarray: Binomial tree matrix representing the evolution of interest rates (e.g., LIBOR) over time, entries as percentages.
            2. np.ndarray: A matrix of the same shape representing the risk-neutral probability of upward movement at each node.
        loan_y (int): Number of years for the loan, matching number of payments over the loan term.
        option_rate (float): The strike or exercise rate of the option, expressed as a percentage.
        p (float): Risk-neutral probability, a value between 0 and 1, representing the likelihood of an upward movement in the binomial tree.

    Returns:
        np.ndarray: A matrix of option values, representing the option value at each node, discounted back to time t = 0.
    '''
    # Separate rates matrix and risk-neutral probabilities matrix
    rates_matrix, prob = rates_prob_matrix
    # Number of time steps in the rate tree
    N = rates_matrix.shape[0] - 1
    # Initialize the matrix to store the option values at each node, set to zero initially
    values = np.zeros((N + 1, N + 1))

    # Step 1: Calculate option values at maturity
    # Loop through each node at maturity (last column in the matrix)
    for i in range(N):
        # If the interest rate at this node is higher than the option's rate, the option has value > 0
        if rates_matrix[i, N] > option_rate:
            sum = 0
            # Loop through each year of the loan (number of payments)
            for j in range(1, loan_y + 1):
                # Calculate the discounted cashflow for this year using the discount factor function (d_f)
                sum += d_f(N + j, N) * (rates_matrix[i, N] - option_rate) / 100
            # Store the value at this node, considering the discount factor at time 0
            values[i, N] = sum * d_f(N, 0)

    # Step 2: Do backwards induction to calculate option values for earlier time steps
    # Loop through all time steps in reverse
    for i in range(N - 1, -1, -1):
        # Loop through each node at this time step
        for j in range(i + 1):
            sum = 0
            # If the interest rate at this node is greater than the option rate, the option has immediate exercise value > 0
            if rates_matrix[j, i] > option_rate:
                # Loop through each year of the loan (number of payments)
                for k in range(1, loan_y + 1):
                    # Calculate the discounted cashflow for each year, using the discount factor function (d_f)
                    sum += d_f(i + k, i) * (rates_matrix[j, i] - option_rate) / 100
            # At each node, compute the maximum of:
            # 1. The immediate exercise value (if exercising the option is advantageous)
            # 2. The expected value of holding the option (discounted future values from the next time step)
            values[j, i] = max([sum * d_f(i, 0), prob[j, i] * values[j, i + 1] + (1 - prob[j, i]) * values[j + 1, i + 1]])

    return values


# Example parameters
r0 = 4
u = 1.1
N = 5
loan_y = 20
option_rate = 4
interest_rate_tree = binomial_interest_rate_tree(r0, u, N)

print(interest_rate_tree[0])
print(interest_rate_tree[1])
print(np.round(option_values(interest_rate_tree, loan_y, option_rate), 5))

[[4.         4.4        4.84       5.324      5.8564     6.44204   ]
 [0.         3.63636364 4.         4.4        4.84       5.324     ]
 [0.         0.         3.30578512 3.63636364 4.         4.4       ]
 [0.         0.         0.         3.0052592  3.30578512 3.63636364]
 [0.         0.         0.         0.         2.73205382 3.0052592 ]
 [0.         0.         0.         0.         0.         2.48368529]]
[[0.5        0.43644054 0.36843543 0.29819465 0.22887406 0.16433013]
 [0.         0.557824   0.5        0.43644054 0.36843543 0.29819465]
 [0.         0.         0.60937328 0.557824   0.5        0.43644054]
 [0.         0.         0.         0.65464669 0.60937328 0.557824  ]
 [0.         0.         0.         0.         0.6939852  0.65464669]
 [0.         0.         0.         0.         0.         0.7279172 ]]
[[0.04367 0.06512 0.10555 0.15996 0.21566 0.27278]
 [0.      0.02222 0.03382 0.05518 0.09758 0.14789]
 [0.      0.      0.00759 0.01246 0.02234 0.04468]
 [0.      0.     